# Persisting AMFabric

In [ ]:
from src.amfabric import AMFabric
from src.sdr import SDR


# library to plot data
#
import plotly.graph_objects as go


# plot the fabric structure
#
def plot_fabric_3d(fabric, title, coords_to_highlight=None, neurons_to_plot=None, short_term_memory=1):
    x_node = []
    y_node = []
    z_node = []
    colors = []
    x_edge = []
    y_edge = []
    z_edge = []
    labels = []
    sizes = []
    pairs = set()
    nos_bmus = 0
    
    if neurons_to_plot is None:
        neurons_to_plot = [n for n in range(short_term_memory)]
    
    if coords_to_highlight is None:
        coords_to_highlight = list(fabric['neuro_columns'].keys())
        
    for coord_id in fabric['neuro_columns'].keys():

        if fabric['neuro_columns'][coord_id]['n_bmu'] > 0:
            nos_bmus += 1

        for neuron_id in neurons_to_plot:
            
            # if there are edges for this neuron_id
            #
            if sum([1 for edge in fabric['neuro_columns'][coord_id]['neuro_column'] if fabric['neuro_columns'][coord_id]['neuro_column'][edge]['neuron_id'] == neuron_id]) > 0:
            
                # if neuron in the top of the column and has been trained then increase size
                #
                if neuron_id == neurons_to_plot[0] and fabric['neuro_columns'][coord_id]['n_bmu'] + fabric['neuro_columns'][coord_id]['n_nn'] > 0:
                    sizes.append(20 + fabric['neuro_columns'][coord_id]['n_bmu'])
                else:
                    sizes.append(15)

                nc = fabric['neuro_columns'][coord_id]['neuro_column']

                x = fabric['neuro_columns'][coord_id]['coord'][0]
                y = fabric['neuro_columns'][coord_id]['coord'][1]
                z = max(neurons_to_plot) - neuron_id

                x_node.append(x)
                y_node.append(y)
                z_node.append(z)

                if coord_id in coords_to_highlight:
                    r = round(nc['trade:*:{}:{}:{}:{}:{}'.format('has_rgb', 'r', neuron_id, 'rgb', 'r')]['numeric'])
                    g = round(nc['trade:*:{}:{}:{}:{}:{}'.format('has_rgb', 'g', neuron_id, 'rgb', 'g')]['numeric'])
                    b = round(nc['trade:*:{}:{}:{}:{}:{}'.format('has_rgb', 'b', neuron_id, 'rgb', 'b')]['numeric'])
                    opacity = 1.0
                else:
                    r = 255
                    g = 255
                    b = 255
                    opacity = 0.7
                colors.append('rgba({},{},{},{})'.format(r, g, b, opacity))

                txt = 'NeuroColumn: {}<br>Neuron: {}<br>n_BMU: {}<br>last_bmu: {}<br>n_nn: {}<br>last_nn: {}<br>mean_dist: {:.2f}<br>mean_sim: {:.2f}<br>community:{}'.format(coord_id, 
                                                                                                                           neuron_id, 
                                                                                                                           fabric['neuro_columns'][coord_id]['n_bmu'],
                                                                                                                           fabric['neuro_columns'][coord_id]['last_bmu'],
                                                                                                                           fabric['neuro_columns'][coord_id]['n_nn'],
                                                                                                                           fabric['neuro_columns'][coord_id]['last_nn'],
                                                                                                                           fabric['neuro_columns'][coord_id]['mean_distance'] if 'mean_distance' in fabric['neuro_columns'][coord_id] else None,
                                                                                                                           fabric['neuro_columns'][coord_id]['mean_similarity'] if 'mean_similarity' in fabric['neuro_columns'][coord_id] else None,
                                                                                                                           fabric['neuro_columns'][coord_id]['community'],
                                                                                                                           )
                edges = list(fabric['neuro_columns'][coord_id]['neuro_column'].keys())
                edges.sort()
                for edge in edges:
                    if fabric['neuro_columns'][coord_id]['neuro_column'][edge]['neuron_id'] == neuron_id:
                        
                        # get the numeric if it exists
                        #
                        if 'numeric' in fabric['neuro_columns'][coord_id]['neuro_column'][edge]:
                            numeric = '{:.2f}'.format(fabric['neuro_columns'][coord_id]['neuro_column'][edge]['numeric'])
                        else:
                            numeric = None
                        
                        txt = '{}<br>{}: {} {}: {} Prob: {:.2f} Numeric: {}'.format(txt, 
                                                                              fabric['neuro_columns'][coord_id]['neuro_column'][edge]['edge_type'],
                                                                              fabric['neuro_columns'][coord_id]['neuro_column'][edge]['edge_uid'],
                                                                              fabric['neuro_columns'][coord_id]['neuro_column'][edge]['target_type'],
                                                                              fabric['neuro_columns'][coord_id]['neuro_column'][edge]['target_uid'],
                                                                              fabric['neuro_columns'][coord_id]['neuro_column'][edge]['prob'],
                                                                              numeric
                                                                             )
                labels.append(txt)

                # connect neurons in same column
                #
                if neuron_id < neurons_to_plot[-1]:
                    pair = (min((x, y, neuron_id), (x, y, neuron_id + 1)), max((x, y, neuron_id), (x, y, neuron_id + 1)))

                    pairs.add(pair)

                    x_edge.append(x)
                    x_edge.append(x)
                    x_edge.append(None)

                    y_edge.append(y)
                    y_edge.append(y)
                    y_edge.append(None)

                    nn_z = max(neurons_to_plot) - neuron_id - 1
                    z_edge.append(z)
                    z_edge.append(nn_z)
                    z_edge.append(None)

                # connect neuro_columns
                #
                for nn_id in fabric['neuro_columns'][coord_id]['nn']:
                    nn_x = fabric['neuro_columns'][nn_id]['coord'][0]
                    nn_y = fabric['neuro_columns'][nn_id]['coord'][1]

                    pair = (min((x, y, neuron_id), (nn_x, nn_y, neuron_id)), max((x, y, neuron_id), (nn_x, nn_y, neuron_id)))

                    if pair not in pairs:
                        pairs.add(pair)
                        nn_z = max(neurons_to_plot) - neuron_id
                        x_edge.append(x)
                        x_edge.append(nn_x)
                        x_edge.append(None)

                        y_edge.append(y)
                        y_edge.append(nn_y)
                        y_edge.append(None)

                        z_edge.append(z)
                        z_edge.append(nn_z)
                        z_edge.append(None)

    neuron_scatter = go.Scatter3d(x=x_node, y=y_node, z=z_node, hovertext=labels, mode='markers', marker=dict(size=sizes, color=colors, opacity=1.0))

    amf_edge_scatter = go.Scatter3d(x=x_edge, y=y_edge, z=z_edge, mode='lines', line=dict(width=0.5, color='grey'))

    fig = go.Figure(data=[neuron_scatter, amf_edge_scatter])
    fig.update_layout(width=1200, height=1200, title=dict(text=title),
                      scene=dict(xaxis_title='X Coord', yaxis_title='Y Coord', zaxis_title='Sequence'))
    
    print('Nos Neuron_columns: {} Nos BMUs: {}, Nos Mapped: {} Mean Distance: {:.2f} Mean similarity: {:.2f}'.format(len(fabric['neuro_columns']),
                                                                                                                    nos_bmus,
                                                                                                                    fabric['mapped'],
                                                                                                                    fabric['mean_distance'],
                                                                                                                    fabric['mean_similarity']))
    
    fig.show()
    
def explain_search_por(search_por):
    
    total_dist = sum([search_por[neuron]['distance'] for neuron in search_por])
    neuron_dist = [[neuron, search_por[neuron]['distance']] for neuron in search_por]
    neuron_dist.sort(key=lambda x: x[1], reverse=True)
    
    print('Total Distance: {:.2f} Distance contribution by neuron:'.format(total_dist))
    for n in neuron_dist:
        print('  Neuron: {} Distance: {:.2f} %Distance: {:.2%}'.format(n[0], n[1], n[1] / total_dist))
    
    for neuron in search_por:
        total_dist = search_por[neuron]['distance']
        edge_dist = [(edge, search_por[neuron]['edges'][edge]['distance']) for edge in search_por[neuron]['edges']]
        print('')
        edge_dist.sort(key=lambda x: x[1], reverse=True)
        for edge in edge_dist:
            print('      Edge: {} Distance: {:.2f} %Distance: {:.2%}'.format(edge[0], edge[1], edge[1] / total_dist))
            edge_data = search_por[neuron]['edges'][edge[0]]
            print('        Prob distance: {:.2f} neuro_column: {:.2f} sdr: {:.2f}'.format(edge_data['prob']['distance'],edge_data['prob']['nc'],edge_data['prob']['compare_nc']))
            if 'numeric' in edge_data:
                print('        Numeric distance: {:.2f} neuro_column: {:.2f} sdr: {:.2f}'.format(edge_data['numeric']['distance'],edge_data['numeric']['nc'],edge_data['prob']['compare_nc']))
            print('')
        
    return neuron_dist


### Instantiate the Fabric

In [ ]:
# the length of sequence to hold in short term memory
#
short_term_memory = 1

# lookback window threshold for detecting anomalies and motifs
#
mp_threshold = 0.10

# connection topology of neur_column in amfabric
#
structure = 'star'

# edges with probabilities below this threshold will be assumed zero and deleted
#
prune_threshold = 0.001

# a random seed used when growing the fabric and initialising neuro columns
#
random_seed = 221166

# instantiate an area of the amfabric for the selected client
#
amf = AMFabric(uid='client_x_trades',
               short_term_memory=short_term_memory,
               mp_threshold=mp_threshold,
               structure=structure,
               prune_threshold=prune_threshold,
               min_cluster_size=2,
               cluster_start_threshold=0.5,
               cluster_step=0.01,
               random_seed=random_seed)

# a list to collect the path of reasoning data
#
por_results = []

### Train with first SDR

In [ ]:
sdr_1 = SDR()

sdr_1.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'r'),
               target_node=('rgb', 'r'),
               probability=1.0,
               numeric=125,
               numeric_min=0,
               numeric_max=255
               )

sdr_1.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'g'),
               target_node=('rgb', 'g'),
               probability=1.0,
               numeric=125,
               numeric_min=0,
               numeric_max=255
               )

sdr_1.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'b'),
               target_node=('rgb', 'b'),
               probability=1.0,
               numeric=125,
               numeric_min=0,
               numeric_max=255
               )

    
por = amf.train(sdr=sdr_1, ref_id=1, fast_search=True, similarity_learn=True)
por_results.append(por)

# extract the current fabric
#
fabric = amf.decode_fabric(all_details=True)

# print specific level
#
plot_fabric_3d(fabric, 'client_x_trades', coords_to_highlight=None, neurons_to_plot=[0], short_term_memory=short_term_memory)


### Path of Reasoning data is also created

In [ ]:
# print out POR of last trade
#
por_idx = -1
for attr in por_results[por_idx]:
    if attr not in ['fabric_distance']:
        print(attr, por_results[por_idx][attr])

# print out POR for Search calc for BMU NeuroColumn 
print('')
print('dist for BMU coord:', por_results[por_idx]['bmu'])
for attr in por_results[por_idx]['fabric_distance'][por_results[por_idx]['bmu']]:
    if attr != 'por':
        print(attr,':', por_results[por_idx]['fabric_distance'][por_results[por_idx]['bmu']][attr])
    else:
        explain_search_por(search_por=por_results[por_idx]['fabric_distance'][por_results[por_idx]['bmu']]['por'])

### Extract a "Persist Graph"

In [ ]:
persist_graph = amf.get_persist_graph(ref_id=1)
print(persist_graph)

In [ ]:
persist_graph.plot(dimension=2, width=1700, height=1000)

In [ ]:
import os
from dotenv import load_dotenv
from src.kv_cache import KVGraphCache

load_dotenv()

config = {'db_name': os.getenv("DB_NAME"),
          'db_username': os.getenv("DB_USERNAME"),
          'db_password': os.getenv("DB_PASSWORD"),
          'db_system': os.getenv("DB_SYSTEM"),
          'db_config_file_path': os.getenv("DB_CONFIG_PATH"),
          'db_queries_file_path': os.getenv("DB_QUERIES_PATH"),
          }

in_memory_cache = KVGraphCache(config=config)

# save to cache
#
in_memory_cache.set_kv(store_name='amfabric', key='client_x_trades', value=persist_graph)

# ask cache to persist
#
in_memory_cache.persist()

### Train next SDR

In [ ]:
sdr_2 = SDR()
sdr_2.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'r'),
               target_node=('rgb', 'r'),
               probability=1.0,
               numeric=200,
               numeric_min=0,
               numeric_max=255
               )

sdr_2.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'g'),
               target_node=('rgb', 'g'),
               probability=1.0,
               numeric=200,
               numeric_min=0,
               numeric_max=255
               )

sdr_2.set_item(source_node=('trade', '*'),
               edge=('has_rgb', 'b'),
               target_node=('rgb', 'b'),
               probability=1.0,
               numeric=200,
               numeric_min=0,
               numeric_max=255
               )
    
por = amf.train(sdr=sdr_2, ref_id=2, fast_search=True, similarity_learn=True)
por_results.append(por)

# extract the current fabric
#
fabric = amf.decode_fabric(all_details=True)

# print specific level
#
plot_fabric_3d(fabric, 'client_x_trades', coords_to_highlight=None, neurons_to_plot=[0], short_term_memory=short_term_memory)


### Update existing persist graph and save to cache

In [ ]:
persist_graph = amf.get_persist_graph(ref_id=2)
print(persist_graph)

In [ ]:
in_memory_cache.set_kv(store_name='amfabric', key='client_x_trades', value=persist_graph)
in_memory_cache.persist()